In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import sys; sys.path.insert(0, '..') # add parent folder path where lib folder is

In [3]:
# from ml.preprocess import data
import pickle
from dask_ml.preprocessing import StandardScaler
from dask_ml.model_selection import train_test_split

from sklearn.metrics import r2_score, mean_squared_error
from sklearn.metrics import f1_score, roc_auc_score, precision_score, recall_score

import numpy as np
import pandas as pd

from utils import helper, config, dasker

from ml.models.base.slug_ann import SlugANN
from ml.models.base.slug_xgboost import SlugXGBoost
from ml.models.base.brisk_xgboost import BriskXGBoost
from ml.models.ensemble import BaseUnify


In [4]:
X, y = helper.get_covid_dataset()
# X = X.drop(['location'], axis = 1)
df_X_scalar = StandardScaler().fit_transform(X)     
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33)

In [5]:
client = dasker.get_dask_client()

2022-10-14 07:42:58,882 - distributed.diskutils - INFO - Found stale lock file and directory '/tmp/dask-worker-space/worker-jebkeawt', purging
2022-10-14 07:42:58,884 - distributed.diskutils - INFO - Found stale lock file and directory '/tmp/dask-worker-space/worker-i_ihzxuw', purging
2022-10-14 07:42:58,884 - distributed.diskutils - INFO - Found stale lock file and directory '/tmp/dask-worker-space/worker-0xpvo1pn', purging
2022-10-14 07:42:58,884 - distributed.diskutils - INFO - Found stale lock file and directory '/tmp/dask-worker-space/worker-ko7k51bn', purging


In [6]:
ex = Explainable(df_X_scalar, y, client = client)

In [7]:
ex.load_models()

XGBoostError: [07:43:06] /mnt/c/Users/rwmas/xgboost/src/learner.cc:1066: Check failed: header == serialisation_header_: 
  If you are loading a serialized model (like pickle in Python, RDS in R) generated by
  older XGBoost, please export the model by calling `Booster.save_model` from that version
  first, then load it back in current version. See:

    https://xgboost.readthedocs.io/en/latest/tutorials/saving_model.html

  for more details about differences between saving model and serializing.

Stack trace:
  [bt] (0) /home/vagrant/miniconda3/envs/test/lib/python3.8/site-packages/xgboost/lib/libxgboost.so(+0x2c27b8) [0x7fd8027327b8]
  [bt] (1) /home/vagrant/miniconda3/envs/test/lib/python3.8/site-packages/xgboost/lib/libxgboost.so(xgboost::LearnerIO::Load(dmlc::Stream*)+0x2a4) [0x7fd802756ee4]
  [bt] (2) /home/vagrant/miniconda3/envs/test/lib/python3.8/site-packages/xgboost/lib/libxgboost.so(XGBoosterUnserializeFromBuffer+0x54) [0x7fd802588b14]
  [bt] (3) /home/vagrant/miniconda3/env

In [78]:
attr_algos = ['IG', 'SHAP', 'GradientSHAP']
res = ex.get_attr(attr_algos=['IG', 'SHAP', 'GradientSHAP'])

calculating variable importance on Sequential(
  (0): Linear(in_features=41, out_features=5, bias=True)
  (1): ReLU()
  (2): Linear(in_features=5, out_features=173, bias=True)
  (3): ReLU()
  (4): Linear(in_features=173, out_features=139, bias=True)
  (5): ReLU()
  (6): Linear(in_features=139, out_features=55, bias=True)
  (7): ReLU()
  (8): Linear(in_features=55, out_features=18, bias=True)
  (9): ReLU()
  (10): Linear(in_features=18, out_features=4, bias=True)
  (11): ReLU()
  (12): Linear(in_features=4, out_features=99, bias=True)
  (13): ReLU()
  (14): Linear(in_features=99, out_features=1, bias=True)
  (15): ReLU()
)
calculating variable importance on Sequential(
  (0): Linear(in_features=41, out_features=164, bias=True)
  (1): ReLU()
  (2): Linear(in_features=164, out_features=214, bias=True)
  (3): ReLU()
  (4): Linear(in_features=214, out_features=1, bias=True)
  (5): ReLU()
)


In [79]:
res

,IG_model_1,SHAP_model_1,GradientSHAP_model_1,IG_model_2,SHAP_model_2,GradientSHAP_model_2
location,0.002277,0.003038,0.004303,0.0,0.0,0.0
new_cases_per_million,0.002693,0.003556,0.004102,0.0,0.0,0.0
population_cov,0.000228,0.000263,0.000467,0.0,0.0,0.0
life_expectancy_cov,0.000350,0.000542,0.000949,0.0,0.0,0.0
Population,0.000330,0.000437,0.000509,0.0,0.0,0.0
Area_km2,0.000924,0.000953,0.001831,0.0,0.0,0.0
Density_km2,0.000228,0.000143,0.000408,0.0,0.0,0.0
Year_x,0.000000,0.000000,0.000203,0.0,0.0,0.0
Meningitis,0.000471,0.000482,0.000587,0.0,0.0,0.0
Neoplasms,0.000916,0.001300,0.001725,0.0,0.0,0.0


In [50]:
# res
df_results = pd.DataFrame(res).T

df_results.index = X.columns


In [69]:

cols = []
model_count = 0
for model_count in range(1, len(ex.models)+1):
    #model_count = model_count + 1
    for algo in attr_algos:
        col_name = algo + '_model_' +str(model_count)
        cols.append(col_name)

cols

['IG_model_1',
 'SHAP_model_1',
 'GradientSHAP_model_1',
 'IG_model_2',
 'SHAP_model_2',
 'GradientSHAP_model_2']

In [71]:
df_results.columns = cols
df_results

,IG_model_1,SHAP_model_1,GradientSHAP_model_1,IG_model_2,SHAP_model_2,GradientSHAP_model_2
location,0.002277,0.003337,0.004634,0.0,0.0,0.0
new_cases_per_million,0.002693,0.003707,0.004164,0.0,0.0,0.0
population_cov,0.000228,0.000253,0.000458,0.0,0.0,0.0
life_expectancy_cov,0.000350,0.000552,0.000844,0.0,0.0,0.0
Population,0.000330,0.000455,0.000506,0.0,0.0,0.0
Area_km2,0.000924,0.001183,0.001237,0.0,0.0,0.0
Density_km2,0.000228,0.000163,0.000409,0.0,0.0,0.0
Year_x,0.000000,0.000000,0.000227,0.0,0.0,0.0
Meningitis,0.000471,0.000512,0.000647,0.0,0.0,0.0
Neoplasms,0.000916,0.001408,0.001862,0.0,0.0,0.0
